# 01 – Stain Normalization & Cell Segmentation

This notebook performs:
1. **Stain normalization** using Macenko/Vahadane method (automated, no manual region selection)
2. **Cell segmentation** using InstanSeg or classical methods
3. **Marker detection** for insulin, glucagon, CD3 (for fluorescence images)
4. **Comprehensive visualizations** with full-image and close-up views

Supports both brightfield and fluorescent multiplex images.

In [ ]:
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Allow very large whole-slide images
os.environ.setdefault('OPENCV_IO_MAX_IMAGE_PIXELS', str(2**63 - 1))

import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from skimage import exposure

try:
    import openslide
    _OPENSLIDE_AVAILABLE = True
except ImportError:
    openslide = None
    _OPENSLIDE_AVAILABLE = False
    print('OpenSlide not available; WSI formats may not load')

from isletscope.stain import StainNormalizer
from isletscope.segmentation import CellSegmenter

print('Imports complete')

## Configuration

In [ ]:
# ===== Input Configuration =====
image_path = '../images/129753.svs'  # Update to your file
max_dimension = 12000  # Downsample WSI to this max dimension
image_type = 'brightfield'  # 'brightfield' or 'fluorescence'

# ===== Stain Normalization =====
use_stain_norm = True
stain_method = 'macenko'  # 'macenko' or 'vahadane'
use_gpu = False  # Set True if CuPy installed

# ===== Segmentation =====
backend = 'instanseg'  # 'instanseg', 'model', or 'classical'
probability_threshold = 0.5
min_cell_size = 32  # Remove objects smaller than this

# ===== InstanSeg Parameters =====
# Model selection
instanseg_model = 'brightfield_nuclei'  # 'brightfield_nuclei' or 'fluorescence_nuclei_and_cells'

# Tiling parameters for large images (important for WSI)
tile_size = 1024  # Tile size in pixels (512, 1024, or 2048)
tile_overlap = 64  # Overlap between tiles (prevents edge artifacts)

# Performance parameters
batch_size = 4  # Number of tiles to process in parallel (GPU-dependent)
pixel_size = None  # Physical pixel size in microns (auto-detected if None)
normalization = True  # Apply intensity normalization
image_reader = 'tiffslide'  # Image reading backend ('tiffslide', 'openslide')

# ===== Marker Detection (for fluorescence) =====
marker_channels = {'insulin': 0, 'glucagon': 1, 'CD3': 2}
marker_thresholds = {'insulin': 80, 'glucagon': 80, 'CD3': 40}

# ===== Visualization =====
# Closeup regions (y_start, y_end, x_start, x_end) as fractions of image dimensions
closeup_regions = [
    (0.3, 0.4, 0.5, 0.6),  # Region 1
    (0.6, 0.7, 0.7, 0.8),  # Region 2
]

# ===== Output =====
output_dir = Path('../outputs')
output_dir.mkdir(exist_ok=True)

print('Configuration set')

## Image Loading

In [ ]:
def load_image(path: str, max_dim: int = 2000):
    """Load image; for WSI use OpenSlide at downsampled level."""
    p = Path(path)
    suffix = p.suffix.lower()
    
    if _OPENSLIDE_AVAILABLE and suffix in {'.svs', '.tif', '.tiff', '.ndpi', '.scn'}:
        slide = openslide.OpenSlide(str(p))
        level = len(slide.level_dimensions) - 1
        for i, (w, h) in enumerate(slide.level_dimensions):
            if max(w, h) <= max_dim:
                level = i
                break
        region = slide.read_region((0, 0), level, slide.level_dimensions[level])
        img = cv2.cvtColor(np.array(region.convert('RGB')), cv2.COLOR_RGB2BGR)
        slide.close()
        return img
    
    img = cv2.imread(str(p))
    if img is None:
        raise FileNotFoundError(f'Cannot load image: {path}')
    return img

def get_closeup_coords(img_shape, region_frac):
    """Convert fractional coordinates to pixel coordinates."""
    h, w = img_shape[:2]
    y1, y2, x1, x2 = region_frac
    return (int(y1*h), int(y2*h), int(x1*w), int(x2*w))

print('Helper functions defined')

In [ ]:
img_original = load_image(image_path, max_dim=max_dimension)
print(f'Loaded image: {img_original.shape} ({img_original.dtype})')
print(f'Image size: {img_original.shape[1]} x {img_original.shape[0]} pixels')
print(f'Memory: {img_original.nbytes / 1e6:.1f} MB')

## Stain Normalization

Automated stain vector estimation and normalization (no manual region selection required).

In [ ]:
if use_stain_norm and image_type == 'brightfield':
    print(f'Running {stain_method} normalization...')
    normalizer = StainNormalizer(method=stain_method, use_gpu=use_gpu)
    normalizer.estimate_stain_matrix(img_original)
    img_normalized = normalizer.normalize(img_original)
    print('Stain normalization complete')
else:
    img_normalized = img_original.copy()
    print('Skipping stain normalization')

In [ ]:
# Full image comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
axes[0].imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB))
axes[1].set_title('Stain Normalized', fontsize=14, fontweight='bold')
axes[1].axis('off')

# Draw rectangles showing closeup regions
for i, region_frac in enumerate(closeup_regions):
    y1, y2, x1, x2 = get_closeup_coords(img_original.shape, region_frac)
    rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='red', facecolor='none')
    axes[1].add_patch(rect)
    axes[1].text(x1, y1-10, f'Region {i+1}', color='red', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / '01_stain_normalization_full.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Closeup comparison
n_regions = len(closeup_regions)
fig, axes = plt.subplots(n_regions, 2, figsize=(12, 6*n_regions))
if n_regions == 1:
    axes = axes.reshape(1, -1)

for i, region_frac in enumerate(closeup_regions):
    y1, y2, x1, x2 = get_closeup_coords(img_original.shape, region_frac)
    
    orig_crop = img_original[y1:y2, x1:x2]
    norm_crop = img_normalized[y1:y2, x1:x2]
    
    axes[i, 0].imshow(cv2.cvtColor(orig_crop, cv2.COLOR_BGR2RGB))
    axes[i, 0].set_title(f'Region {i+1} - Original', fontsize=12, fontweight='bold')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(cv2.cvtColor(norm_crop, cv2.COLOR_BGR2RGB))
    axes[i, 1].set_title(f'Region {i+1} - Normalized', fontsize=12, fontweight='bold')
    axes[i, 1].axis('off')

plt.tight_layout()
plt.savefig(output_dir / '01_stain_normalization_closeup.png', dpi=150, bbox_inches='tight')
plt.show()

## Cell Segmentation

Segment individual cells/nuclei using InstanSeg or classical methods.

In [ ]:
print(f'Running {backend} segmentation...')
segmenter = CellSegmenter(
    backend=backend,
    use_instanseg=(backend == 'instanseg'),
    probability_threshold=probability_threshold,
    min_size=min_cell_size,
    # InstanSeg-specific parameters
    instanseg_model_name=instanseg_model,
    tile_size=tile_size,
    tile_overlap=tile_overlap,
    batch_size=batch_size,
    pixel_size=pixel_size,
    normalization=normalization,
    image_reader=image_reader,
)

seg_result = segmenter.segment(img_normalized, image_type=image_type)
cell_mask = seg_result['mask']
cell_labels = seg_result['labels']

n_cells = cell_labels.max()
print(f'Segmentation complete: {n_cells:,} cells detected')
print(f'Total cell area: {cell_mask.sum():,} pixels ({100*cell_mask.sum()/cell_mask.size:.1f}% of image)')

In [ ]:
# Marker detection (fluorescence only)
markers = {}
if image_type == 'fluorescence' and marker_channels:
    print('Detecting markers...')
    markers = segmenter.detect_markers(
        img_normalized,
        cell_labels,
        marker_channels=marker_channels,
        thresholds=marker_thresholds,
        brighter_is_positive=True
    )
    for marker, mask in markers.items():
        print(f'  {marker}: {int(mask.sum())} positive cells')
else:
    print('Skipping marker detection (brightfield image)')

In [ ]:
# Full image segmentation overlay
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Original/normalized
axes[0].imshow(cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB))
axes[0].set_title('Normalized Image', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Cell mask
axes[1].imshow(cell_mask, cmap='gray')
axes[1].set_title(f'Cell Mask ({n_cells} cells)', fontsize=14, fontweight='bold')
axes[1].axis('off')

# Overlay
overlay = cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB).copy()
overlay[cell_mask > 0] = overlay[cell_mask > 0] * 0.6 + np.array([255, 0, 0]) * 0.4
axes[2].imshow(overlay)
axes[2].set_title('Segmentation Overlay', fontsize=14, fontweight='bold')
axes[2].axis('off')

# Draw closeup region boxes
for i, region_frac in enumerate(closeup_regions):
    y1, y2, x1, x2 = get_closeup_coords(img_original.shape, region_frac)
    rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='yellow', facecolor='none')
    axes[2].add_patch(rect)
    axes[2].text(x1, y1-10, f'Region {i+1}', color='yellow', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / '02_cell_segmentation_full.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Closeup segmentation views
fig, axes = plt.subplots(n_regions, 3, figsize=(15, 5*n_regions))
if n_regions == 1:
    axes = axes.reshape(1, -1)

for i, region_frac in enumerate(closeup_regions):
    y1, y2, x1, x2 = get_closeup_coords(img_original.shape, region_frac)
    
    img_crop = img_normalized[y1:y2, x1:x2]
    mask_crop = cell_mask[y1:y2, x1:x2]
    labels_crop = cell_labels[y1:y2, x1:x2]
    
    # Image
    axes[i, 0].imshow(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB))
    axes[i, 0].set_title(f'Region {i+1} - Image', fontsize=12, fontweight='bold')
    axes[i, 0].axis('off')
    
    # Labels (colored)
    from skimage.color import label2rgb
    labels_colored = label2rgb(labels_crop, bg_label=0)
    axes[i, 1].imshow(labels_colored)
    axes[i, 1].set_title(f'Region {i+1} - Cell Labels ({labels_crop.max()} cells)', fontsize=12, fontweight='bold')
    axes[i, 1].axis('off')
    
    # Overlay with boundaries
    from skimage.segmentation import find_boundaries
    overlay_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB).copy()
    boundaries = find_boundaries(labels_crop, mode='thick')
    overlay_crop[boundaries] = [255, 255, 0]  # Yellow boundaries
    axes[i, 2].imshow(overlay_crop)
    axes[i, 2].set_title(f'Region {i+1} - Cell Boundaries', fontsize=12, fontweight='bold')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.savefig(output_dir / '02_cell_segmentation_closeup.png', dpi=150, bbox_inches='tight')
plt.show()

## Save Results

In [ ]:
# Save outputs for next notebook
np.save(output_dir / 'img_normalized.npy', img_normalized)
np.save(output_dir / 'cell_mask.npy', cell_mask)
np.save(output_dir / 'cell_labels.npy', cell_labels)

if markers:
    for marker, mask in markers.items():
        np.save(output_dir / f'marker_{marker}.npy', mask)

print(f'Results saved to {output_dir}')
print('\nReady for notebook 02: Islet Detection & Radial Analysis')